# Criação da Camada Ouro - Deduplicação e Cruzamento

Este notebook cria a Camada Ouro (Golden Record) através da deduplicação e cruzamento de dados de múltiplas fontes (DNE, CNEFE, OSM).

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import hashlib

# Schema esperado para dados de fontes (DNE, CNEFE, OSM)
schema_fonte = StructType([
    StructField("id_fonte", StringType(), True),
    StructField("fonte", StringType(), True),  # DNE, CNEFE, OSM
    StructField("tipo_logradouro", StringType(), True),
    StructField("nome_logradouro", StringType(), True),
    StructField("numero", StringType(), True),
    StructField("bairro", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("cep", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("confiabilidade", DoubleType(), True),  # Score de confiabilidade da fonte
])

In [ ]:
# UDF para gerar hash do endereço (usado para deduplicação)
def gerar_hash_endereco(tipo_log, nome_log, numero, bairro, cidade, uf):
    """Gera hash MD5 do endereço para identificação de duplicatas"""
    endereco_completo = f"{tipo_log}|{nome_log}|{numero}|{bairro}|{cidade}|{uf}"
    endereco_normalizado = endereco_completo.upper().strip().replace(" ", "")
    return hashlib.md5(endereco_normalizado.encode('utf-8')).hexdigest()

udf_gerar_hash = udf(gerar_hash_endereco, StringType())

In [ ]:
# Função para carregar e normalizar dados de uma fonte
def carregar_fonte(path_fonte, nome_fonte):
    """
    Carrega dados de uma fonte e adiciona metadados
    
    Args:
        path_fonte: Caminho no MinIO para os dados da fonte
        nome_fonte: Nome da fonte (DNE, CNEFE, OSM)
    
    Returns:
        DataFrame com dados da fonte
    """
    df_fonte = read_delta_table(path_fonte) \
        .withColumn("fonte", lit(nome_fonte)) \
        .withColumn("hash_endereco", 
            udf_gerar_hash(
                col("tipo_logradouro"),
                col("nome_logradouro"),
                col("numero"),
                col("bairro"),
                col("cidade"),
                col("uf")
            )
        )
    
    return df_fonte

In [ ]:
# Função para unificar todas as fontes
def unificar_fontes():
    """
    Carrega e unifica dados de todas as fontes (DNE, CNEFE, OSM)
    
    Returns:
        DataFrame unificado com todas as fontes
    """
    # Carregar cada fonte
    df_dne = carregar_fonte(PATH_DNE, "DNE")
    df_cnefe = carregar_fonte(PATH_CNEFE, "CNEFE")
    df_osm = carregar_fonte(PATH_OSM, "OSM")
    
    # Unificar todas as fontes
    df_unificado = df_dne.unionByName(df_cnefe, allowMissingColumns=True) \
                        .unionByName(df_osm, allowMissingColumns=True)
    
    return df_unificado

In [ ]:
# Função para deduplicar por hash
def deduplicar_por_hash(df_unificado):
    """
    Remove duplicatas exatas usando hash do endereço
    Mantém o registro com maior confiabilidade
    
    Args:
        df_unificado: DataFrame com dados de todas as fontes
    
    Returns:
        DataFrame deduplicado
    """
    window_spec = Window.partitionBy("hash_endereco").orderBy(desc("confiabilidade"), desc("fonte"))
    
    df_deduplicado = df_unificado \
        .withColumn("rank", row_number().over(window_spec)) \
        .filter(col("rank") == 1) \
        .drop("rank")
    
    return df_deduplicado

In [ ]:
# Função para criar registro canônico (Golden Record)
def criar_registro_canonico(df_deduplicado):
    """
    Cria registro canônico agregando informações de múltiplas fontes
    quando há correspondência geográfica
    
    Args:
        df_deduplicado: DataFrame deduplicado
    
    Returns:
        DataFrame com registros canônicos
    """
    # Agrupar por hash e agregar informações
    df_canonico = df_deduplicado \
        .groupBy("hash_endereco", "uf", "cidade", "tipo_logradouro", "nome_logradouro", "numero", "bairro") \
        .agg(
            collect_list("fonte").alias("fontes"),
            avg("latitude").alias("latitude"),
            avg("longitude").alias("longitude"),
            max("confiabilidade").alias("confiabilidade_max"),
            count("*").alias("num_fontes"),
            first("cep").alias("cep"),
            first("id_fonte").alias("id_principal")
        ) \
        .withColumn("uid", 
            concat(
                lit("UID_"),
                col("hash_endereco")
            )
        ) \
        .withColumn("score_confianca", 
            when(col("num_fontes") >= 3, 1.0)  # 3+ fontes concordam
            .when(col("num_fontes") == 2, 0.8)  # 2 fontes concordam
            .otherwise(0.6)  # 1 fonte apenas
        ) \
        .withColumn("criado_em", current_timestamp()) \
        .select(
            col("uid"),
            col("hash_endereco"),
            col("uf"),
            col("cidade"),
            col("tipo_logradouro"),
            col("nome_logradouro"),
            col("numero"),
            col("bairro"),
            col("cep"),
            col("latitude"),
            col("longitude"),
            col("fontes"),
            col("num_fontes"),
            col("score_confianca"),
            col("confiabilidade_max"),
            col("criado_em")
        )
    
    return df_canonico

In [ ]:
# Pipeline completo para criar Camada Ouro
def criar_camada_ouro():
    """
    Pipeline completo para criar a Camada Ouro
    
    Returns:
        DataFrame com Camada Ouro
    """
    print("1. Unificando fontes...")
    df_unificado = unificar_fontes()
    print(f"   Total de registros unificados: {df_unificado.count()}")
    
    print("2. Deduplicando por hash...")
    df_deduplicado = deduplicar_por_hash(df_unificado)
    print(f"   Total após deduplicação: {df_deduplicado.count()}")
    
    print("3. Criando registros canônicos...")
    df_camada_ouro = criar_registro_canonico(df_deduplicado)
    print(f"   Total de registros canônicos: {df_camada_ouro.count()}")
    
    return df_camada_ouro

In [ ]:
# Executar pipeline
# df_camada_ouro = criar_camada_ouro()

# Visualizar amostra
# df_camada_ouro.show(10, truncate=False)

# Estatísticas
# df_camada_ouro.agg(
#     count("*").alias("total_registros"),
#     avg("num_fontes").alias("media_fontes"),
#     avg("score_confianca").alias("score_confianca_medio")
# ).show()

In [ ]:
# Salvar Camada Ouro
# save_delta_table(df_camada_ouro, PATH_CAMADA_OURO, mode="overwrite", partition_by=["uf", "cidade"])

print("Camada Ouro criada com sucesso!")